# Notebook 01 — Pré-processamento dos Dados do CPGF

**Projeto:** Detecção de Anomalias no Uso do Cartão de Pagamento do Governo Federal  
**Disciplina:** FACOM39803 — Mineração de Dados Aplicada a Finanças  
**Etapa:** 1 de 3 — Pré-processamento

---

## 0. Configuração do Ambiente

Montagem do Google Drive (para execução no Colab) e importação das bibliotecas necessárias.

In [ ]:
# --- Montagem do Google Drive (descomente no Colab) ---
# from google.colab import drive
# drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import warnings

warnings.filterwarnings('ignore')
sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (12, 6)

print('Bibliotecas carregadas com sucesso.')

## 1. Carregamento dos Dados

Leitura dos arquivos CSV do CPGF (a partir de 2022) armazenados na pasta `dados/`.

> **Nota:** Os CSVs do Portal da Transparência utilizam separador `;` e encoding `latin-1`.

In [ ]:
# --- Definição de caminhos ---
# Para execução local:
DATA_DIR = os.path.join('..', 'dados')
# Para execução no Colab (descomente e ajuste):
# DATA_DIR = '/content/drive/MyDrive/cpgf-anomaly-detection/dados'

# --- Carregamento de todos os CSVs ---
csv_files = sorted(glob.glob(os.path.join(DATA_DIR, '*.csv')))
print(f'Arquivos encontrados: {len(csv_files)}')
for f in csv_files:
    print(f'  - {os.path.basename(f)}')

# --- Concatenação em um único DataFrame ---
dfs = []
for f in csv_files:
    df_temp = pd.read_csv(f, sep=';', encoding='latin-1')
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)
print(f'\nTotal de registros carregados: {df.shape[0]:,}')
print(f'Total de colunas: {df.shape[1]}')

---

## 2. Exploração dos Dados (Aula 4)

### 2.1 Fundamentação Teórica

A **exploração dos dados** é o primeiro passo fundamental em qualquer projeto de mineração.
Ela tem como objetivo compreender a natureza do conjunto de dados por meio de:

- **Estatísticas resumidas** (medidas de tendência central, dispersão, quartis);
- **Visualizações gráficas** (histogramas, boxplots, gráficos de barras) para identificar
  distribuições, padrões e possíveis anomalias visuais nos atributos.

Essa etapa permite ao analista formular hipóteses iniciais sobre os dados antes de
aplicar qualquer técnica de mineração (Ref.: Aula 4 — Exploração de Dados).

In [ ]:
# --- 2.2 Visão geral do DataFrame ---
print('=== Primeiras linhas ===')
display(df.head())

print('\n=== Informações gerais ===')
df.info()

print('\n=== Tipos de dados ===')
display(df.dtypes)

In [ ]:
# --- 2.3 Estatísticas descritivas ---
print('=== Estatísticas descritivas (atributos numéricos) ===')
display(df.describe())

print('\n=== Estatísticas descritivas (atributos categóricos) ===')
display(df.describe(include='object'))

In [ ]:
# --- 2.4 Visualizações exploratórias ---

# TODO: Ajustar os nomes das colunas conforme a base real do CPGF
# Exemplo de colunas esperadas: 'VALOR_TRANSACAO', 'NOME_ORGAO', 'DATA_TRANSACAO'

# Histograma do valor das transações
# col_valor = 'VALOR_TRANSACAO'  # Ajustar conforme o CSV real
# fig, axes = plt.subplots(1, 2, figsize=(14, 5))
# df[col_valor].hist(bins=50, ax=axes[0], edgecolor='black')
# axes[0].set_title('Distribuição dos Valores de Transação')
# axes[0].set_xlabel('Valor (R$)')
# axes[0].set_ylabel('Frequência')

# Boxplot do valor das transações
# sns.boxplot(x=df[col_valor], ax=axes[1])
# axes[1].set_title('Boxplot — Valores de Transação')
# plt.tight_layout()
# plt.show()

---

## 3. Qualidade dos Dados (Aula 2)

### 3.1 Fundamentação Teórica

A **qualidade dos dados** impacta diretamente os resultados da mineração. Os principais
problemas tratados nesta etapa são:

#### 3.1.1 Tratamento de Ausência de Valores

Valores ausentes (nulos/NaN) podem surgir por falhas de coleta ou campos opcionais.
As estratégias padrão incluem:

- **Eliminar objetos** (linhas) com valores ausentes em atributos críticos;
- **Estimar valores** (imputação) utilizando média, mediana ou moda,
  quando a exclusão geraria perda excessiva de dados.

(Ref.: Aula 2 — Qualidade dos Dados)

#### 3.1.2 Tratamento de Ruído e Dados Inconsistentes

**Ruído** é uma variação aleatória ou erro nos dados medidos (ex: valor negativo
injustificado em uma transação de compra). O tratamento de ruído envolve a limpeza
de atributos com valores visivelmente errados.

> ⚠️ **Distinção crítica:** É fundamental diferenciar **ruído** (erro de coleta/registro
> que deve ser corrigido) de **outlier** (valor extremo genuíno que pode representar
> uma anomalia/fraude financeira — nosso alvo de detecção). O ruído é eliminado
> nesta etapa; os outliers são preservados para análise na Etapa 2.

(Ref.: Aula 2 — Ruído e Dados Inconsistentes)

In [ ]:
# --- 3.2 Análise de valores ausentes ---
print('=== Valores ausentes por coluna ===')
nulos = df.isnull().sum()
nulos_pct = (nulos / len(df)) * 100
resumo_nulos = pd.DataFrame({'Nulos': nulos, '% do Total': nulos_pct.round(2)})
display(resumo_nulos[resumo_nulos['Nulos'] > 0].sort_values('Nulos', ascending=False))

In [ ]:
# --- 3.3 Tratamento de valores ausentes ---

# Estratégia 1: Eliminar colunas com mais de X% de nulos
LIMIAR_NULOS = 70  # percentual
colunas_excluir = resumo_nulos[resumo_nulos['% do Total'] > LIMIAR_NULOS].index.tolist()
print(f'Colunas removidas (> {LIMIAR_NULOS}% nulos): {colunas_excluir}')
df.drop(columns=colunas_excluir, inplace=True, errors='ignore')

# Estratégia 2: Para colunas numéricas restantes, imputar com mediana
cols_numericas = df.select_dtypes(include=[np.number]).columns
for col in cols_numericas:
    if df[col].isnull().sum() > 0:
        mediana = df[col].median()
        df[col].fillna(mediana, inplace=True)
        print(f'  Coluna "{col}": nulos imputados com mediana = {mediana:.2f}')

# Estratégia 3: Para colunas categóricas, imputar com moda ou 'DESCONHECIDO'
cols_categoricas = df.select_dtypes(include='object').columns
for col in cols_categoricas:
    if df[col].isnull().sum() > 0:
        df[col].fillna('DESCONHECIDO', inplace=True)
        print(f'  Coluna "{col}": nulos preenchidos com "DESCONHECIDO"')

print(f'\nValores ausentes restantes: {df.isnull().sum().sum()}')

In [ ]:
# --- 3.4 Remoção de ruído e dados inconsistentes ---

# TODO: Ajustar conforme as colunas reais do CSV
# Exemplo: remover transações com valor = 0 (sem significado)
# col_valor = 'VALOR_TRANSACAO'
# n_antes = len(df)
# df = df[df[col_valor] != 0]
# print(f'Registros com valor 0 removidos: {n_antes - len(df)}')

# Nota: Valores negativos podem representar estornos legítimos ou erros.
# Avaliar e documentar a decisão:
# n_negativos = (df[col_valor] < 0).sum()
# print(f'Transações com valor negativo: {n_negativos}')
# Decisão: manter valores negativos para análise ou remover como ruído?

print('Etapa de remoção de ruído concluída.')
print(f'Shape atual do DataFrame: {df.shape}')

---

## 4. Transformação dos Dados (Aula 2)

### 4.1 Fundamentação Teórica — Normalização

A **normalização** é uma técnica de transformação que consiste em "fazer o conjunto
inteiro de valores de um atributo ter uma propriedade particular" (Ref.: Aula 2 —
Transformação de Dados).

No contexto deste projeto, a normalização dos **atributos financeiros contínuos**
(como valor da transação) é um **passo essencial** antes da etapa de mineração,
pois os algoritmos de agrupamento (DBSCAN e K-Médias) baseiam-se no **cálculo de
distância** entre pontos. Sem normalização, atributos com escalas maiores dominariam
a medida de distância, distorcendo os resultados.

Técnicas comuns de normalização:

- **Min-Max Scaling:** Transforma os valores para o intervalo [0, 1].
- **Z-Score (Standardization):** Centraliza na média 0 e desvio padrão 1.

Utilizaremos **StandardScaler** (Z-Score) neste projeto, pois é mais robusto
na presença de outliers que a normalização Min-Max.

In [ ]:
from sklearn.preprocessing import StandardScaler

# --- 4.2 Seleção de atributos numéricos para normalização ---
cols_para_normalizar = df.select_dtypes(include=[np.number]).columns.tolist()
print(f'Colunas numéricas selecionadas para normalização: {cols_para_normalizar}')

# --- 4.3 Aplicação do StandardScaler (Z-Score) ---
scaler = StandardScaler()
df_normalizado = df.copy()
df_normalizado[cols_para_normalizar] = scaler.fit_transform(df[cols_para_normalizar])

print('\n=== Estatísticas após normalização ===')
display(df_normalizado[cols_para_normalizar].describe().round(4))

---

## 5. Exportação dos Dados Pré-processados

O DataFrame limpo e normalizado é salvo para ser consumido pelo Notebook 02 (Mineração).

In [ ]:
# --- 5.1 Salvar dados processados ---
OUTPUT_DIR = os.path.join('..', 'dados')
# Para Colab:
# OUTPUT_DIR = '/content/drive/MyDrive/cpgf-anomaly-detection/dados'

# DataFrame original limpo (sem normalização) — para análise qualitativa no Notebook 03
df.to_csv(os.path.join(OUTPUT_DIR, 'cpgf_limpo.csv'), index=False, sep=';', encoding='utf-8')
print('Arquivo salvo: cpgf_limpo.csv')

# DataFrame normalizado — para entrada nos algoritmos de agrupamento
df_normalizado.to_csv(os.path.join(OUTPUT_DIR, 'cpgf_normalizado.csv'), index=False, sep=';', encoding='utf-8')
print('Arquivo salvo: cpgf_normalizado.csv')

print(f'\nShape final: {df_normalizado.shape}')
print('\n✅ Pré-processamento concluído com sucesso!')